<a href="https://colab.research.google.com/github/Msingi-AI/Sauti-Ya-Kenya/blob/main/colab/train_tts_fixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sauti Ya Kenya - TTS Training

This notebook sets up the training environment for our Swahili TTS model with TPU support.

In [ ]:
# Mount Google Drive for persistent storage
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Clone repository and install dependencies
!git clone https://github.com/Msingi-AI/Sauti-Ya-Kenya.git
%cd Sauti-Ya-Kenya
!pip install -r requirements.txt

In [ ]:
# Check available hardware
import torch
try:
    import torch_xla
    import torch_xla.core.xla_model as xm
    print("TPU available!")
    DEVICE = xm.xla_device()
    USE_TPU = True
    !pip install cloud-tpu-client==0.10 torch_xla[tpu]>=2.0
except ImportError:
    print("TPU not found, checking for GPU...")
    USE_TPU = False
    if torch.cuda.is_available():
        print(f"GPU available: {torch.cuda.get_device_name(0)}")
        DEVICE = torch.device('cuda')
    else:
        print("No GPU found, using CPU")
        DEVICE = torch.device('cpu')

In [ ]:
# Prepare dataset directory
import os
dataset_dir = "cv-corpus-21.0-delta-2025-03-14/sw"
clips_dir = os.path.join(dataset_dir, "clips")
os.makedirs(clips_dir, exist_ok=True)

# Upload dataset files
from google.colab import files
print("Please upload the following files in order:")
print("1. validated_sentences.tsv")
print("2. clip_durations.tsv")
print("3. All clips_part_*.zip files")

# Handle TSV files first
print("\nUploading TSV files...")
uploaded = files.upload()
for filename in uploaded.keys():
    if filename.endswith('.tsv'):
        dest_path = os.path.join(dataset_dir, filename)
        with open(dest_path, 'wb') as f:
            f.write(uploaded[filename])
        print(f"Saved {filename}")

# Now handle zip files
print("\nUploading and extracting clip files...")
while True:
    try:
        uploaded = files.upload()
        if not uploaded:
            break
            
        for filename in uploaded.keys():
            if filename.startswith('clips_part_') and filename.endswith('.zip'):
                print(f"Extracting {filename}...")
                import zipfile
                with zipfile.ZipFile(filename, 'r') as zip_ref:
                    zip_ref.extractall(dataset_dir)
                os.remove(filename)  # Clean up zip file
                print(f"Extracted {filename}")
    except Exception as e:
        if "No file selected" in str(e):
            break
        else:
            raise e

print("\nDataset preparation complete!")
print(f"Number of audio files: {len(os.listdir(clips_dir))}")

In [ ]:
# Process dataset
!python src/prepare_local_dataset.py \
    --dataset_path "cv-corpus-21.0-delta-2025-03-14/sw" \
    --output_dir "processed_data" \
    --clips_path "clips" \
    --sentences_file "validated_sentences.tsv" \
    --durations_file "clip_durations.tsv"

In [ ]:
# Run data augmentation
!python src/augment_data.py \
    --data_dir "processed_data" \
    --output_dir "augmented_data" \
    --num_augmentations 3

In [ ]:
# Set up training configuration
import json

# Optimize batch size based on hardware
if USE_TPU:
    batch_size = 128  # TPUs handle larger batches efficiently
elif torch.cuda.is_available():
    batch_size = 32   # Standard GPU batch size
else:
    batch_size = 8    # Smaller batch for CPU

config = {
    "train_data_dir": "augmented_data",
    "batch_size": batch_size,
    "learning_rate": 0.001,
    "num_epochs": 100,
    "save_every": 10,
    "device": str(DEVICE),
    "max_len": 10000,  # Based on our Swahili optimization
    "checkpoint_dir": "checkpoints",
    "use_tpu": USE_TPU,
    "gradient_accumulation_steps": 1 if USE_TPU else 2
}

with open('config.json', 'w') as f:
    json.dump(config, f, indent=2)

print(f"Using device: {config['device']}")
print(f"Batch size: {config['batch_size']}")

In [ ]:
# Start training
!python src/train.py --config config.json

In [ ]:
# Run evaluation
!python src/evaluation.py \
    --model_path "checkpoints/best.pt" \
    --test_data "augmented_data/test" \
    --output_dir "evaluation_results"

In [ ]:
# Save model to Google Drive
drive_path = "/content/drive/MyDrive/Sauti-Ya-Kenya/models"
os.makedirs(drive_path, exist_ok=True)

# Copy best model and results
import shutil
shutil.copy("checkpoints/best.pt", f"{drive_path}/best.pt")
shutil.copytree("evaluation_results", f"{drive_path}/evaluation_results", dirs_exist_ok=True)

print("Model and evaluation results saved to Google Drive")